In [1]:
# ----------------------------------
# Implementation
# ----------------------------------

using Distributed
nprocs() == 1 && addprocs(; exeflags = "--project")
workers()

@everywhere using AutoMLPipeline
@everywhere using DataFrames
@everywhere using AutoMLPipeline: score
@everywhere using Random
ENV["COLUMNS"]=1000
ENV["LINES"]=100
# disable truncation of dataframes columns
import Base.show
show(df::AbstractDataFrame) = show(df,truncate=0)
show(io::IO,df::AbstractDataFrame) = show(io,df;truncate=0)

show (generic function with 340 methods)

In [2]:
# brief overview of AutoMLPipeline package
# more details is in my Juliacon2020 presentation about this package
# we use AutoMLPipeline to develop the pipeline optimizer

# define scalers
rb     = SKPreprocessor("RobustScaler",Dict(:name=>"rb"))
pt     = SKPreprocessor("PowerTransformer",Dict(:name=>"pt"))
norm   = SKPreprocessor("Normalizer",Dict(:name=>"norm"))
mx     = SKPreprocessor("MinMaxScaler",Dict(:name=>"mx"))
std    = SKPreprocessor("StandardScaler",Dict(:name=>"std"))
# define extractors
pca    = SKPreprocessor("PCA",Dict(:name=>"pca"))
fa     = SKPreprocessor("FactorAnalysis",Dict(:name=>"fa"))
ica    = SKPreprocessor("FastICA",Dict(:name=>"ica"))
# define learners
rf     = SKLearner("RandomForestClassifier",Dict(:name => "rf"))
ada    = SKLearner("AdaBoostClassifier",Dict(:name => "ada"))
gb     = SKLearner("GradientBoostingClassifier",Dict(:name => "gb"))
lsvc   = SKLearner("LinearSVC",Dict(:name => "lsvc"))
rbfsvc = SKLearner("SVC",Dict(:name => "rbfsvc"))
dt     = SKLearner("DecisionTreeClassifier",Dict(:name =>"dt"))
# preprocessing
noop = Identity(Dict(:name =>"noop"))
ohe  = OneHotEncoder(Dict(:name=>"ohe"))
catf = CatFeatureSelector(Dict(:name=>"catf"))
numf = NumFeatureSelector(Dict(:name=>"numf"))

vscalers =    [rb,pt,norm,mx,std,noop]
vextractors = [pca,fa,ica,noop]
vlearners =   [rf,gb,lsvc,rbfsvc,ada,dt]

dataset = getiris()
X = dataset[:,1:4]
Y = dataset[:,5] |> collect;

In [3]:
# one-block pipeline
prep = @pipeline (catf |> ohe) + numf
ppl1 = @pipeline prep |> (mx |> pca) |> rf
crossvalidate(ppl1,X,Y)

fold: 1, 93.33333333333333
fold: 2, 86.66666666666667
fold: 3, 93.33333333333333
fold: 4, 93.33333333333333
fold: 5, 93.33333333333333
fold: 6, 100.0
fold: 7, 100.0
fold: 8, 100.0
fold: 9, 100.0
fold: 10, 93.33333333333333
errors: 0


(mean = 95.33333333333334, std = 4.499657051403686, folds = 10, errors = 0)

In [4]:
# two-block pipeline
ppl2 = @pipeline prep |> ((norm |> ica) + (pt |> pca)) |> lsvc
crossvalidate(ppl2,X,Y)

fold: 1, 93.33333333333333
fold: 2, 100.0
fold: 3, 100.0
fold: 4, 100.0
fold: 5, 100.0
fold: 6, 100.0
fold: 7, 100.0
fold: 8, 93.33333333333333
fold: 9, 93.33333333333333
fold: 10, 93.33333333333333
errors: 0


(mean = 97.33333333333334, std = 3.442651863295484, folds = 10, errors = 0)

In [5]:
function oneblock_pipeline_factory(scalers,extractors,learners)
   results = @distributed (vcat) for lr in learners
      @distributed (vcat) for xt in extractors
         @distributed (vcat) for sc in scalers
            # baseline preprocessing
            prep = @pipeline ((catf |> ohe) + numf)
            # one-block prp
            expx = @pipeline prep |> (sc |> xt) |> lr
            scn   = sc.name[1:end - 4];xtn = xt.name[1:end - 4]; lrn = lr.name[1:end - 4]
            pname = "($scn |> $xtn) |> $lrn"
            DataFrame(Description=pname,Pipeline=expx)
         end
      end
   end
   return results
end

oneblock_pipeline_factory (generic function with 1 method)

In [6]:
res_oneblock = oneblock_pipeline_factory(vscalers,vextractors,vlearners)

,Description,Pipeline
,String,Pipeline
1,(rb |> pca) |> rf,"Pipeline(""linearpipeline_diU"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_diU"", :machines=>Machine[Pipeline(""linearpipeline_Yky"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_Yky"", :machines=>Workflow[ComboPipeline(""combopipeline_gt1"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_gt1"", :machines=>Machine[Pipeline(""linearpipeline_RGv"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_RGv"", :machines=>Transformer[CatFeatureSelector(""catf_A2A"", Dict{Symbol, Any}(:name=>""catf_A2A"", :nominal_columns=>Int64[])), OneHotEncoder(""ohe_A0W"", Dict{Symbol, Any}(:nominal_column_values_map=>Dict{Int64, Any}(), :name=>""ohe_A0W"", :nominal_columns=>Int64[]))])), NumFeatureSelector(""numf_xzp"", Dict{Symbol, Any}(:name=>""numf_xzp"", :numcols=>Int64[]))])), Pipeline(""linearpipeline_pYm"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_pYm"", :machines=>SKPreprocessor[SKPreprocessor(""rb_HXW"", Dict{Symbol, Any}(:name=>""rb_HXW"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""RobustScaler"")), SKPreprocessor(""pca_1Bi"", Dict{Symbol, Any}(:name=>""pca_1Bi"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PCA""))]))])), SKLearner(""rf_IBN"", Dict{Symbol, Any}(:output=>:class, :name=>""rf_IBN"", :impl_args=>Dict{Symbol, Any}(), :learner=>""RandomForestClassifier""))]))"
2,(pt |> pca) |> rf,"Pipeline(""linearpipeline_9i1"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_9i1"", :machines=>Machine[Pipeline(""linearpipeline_xDI"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_xDI"", :machines=>Workflow[ComboPipeline(""combopipeline_IEu"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_IEu"", :machines=>Machine[Pipeline(""linearpipeline_TtH"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_TtH"", :machines=>Transformer[CatFeatureSelector(""catf_A2A"", Dict{Symbol, Any}(:name=>""catf_A2A"", :nominal_columns=>Int64[])), OneHotEncoder(""ohe_A0W"", Dict{Symbol, Any}(:nominal_column_values_map=>Dict{Int64, Any}(), :name=>""ohe_A0W"", :nominal_columns=>Int64[]))])), NumFeatureSelector(""numf_xzp"", Dict{Symbol, Any}(:name=>""numf_xzp"", :numcols=>Int64[]))])), Pipeline(""linearpipeline_vCF"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_vCF"", :machines=>SKPreprocessor[SKPreprocessor(""pt_IEv"", Dict{Symbol, Any}(:name=>""pt_IEv"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PowerTransformer"")), SKPreprocessor(""pca_1Bi"", Dict{Symbol, Any}(:name=>""pca_1Bi"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PCA""))]))])), SKLearner(""rf_IBN"", Dict{Symbol, Any}(:output=>:class, :name=>""rf_IBN"", :impl_args=>Dict{Symbol, Any}(), :learner=>""RandomForestClassifier""))]))"
3,(norm |> pca) |> rf,"Pipeline(""linearpipeline_HRD"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_HRD"", :machines=>Machine[Pipeline(""linearpipeline_PrC"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_PrC"", :machines=>Workflow[ComboPipeline(""combopipeline_MdH"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_MdH"", :machines=>Machine[Pipeline(""linearpipeline_KlC"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_KlC"", :machines=>Transformer[CatFeatureSelector(""catf_A2A"", Dict{Symbol, Any}(:name=>""catf_A2A"", :nominal_columns=>Int64[])), OneHotEncoder(""ohe_A0W"", Dict{Symbol, Any}(:nominal_column_values_map=>Dict{Int64, Any}(), :name=>""ohe_A0W"", :nominal_columns=>Int64[]))])), NumFeatureSelector(""numf_xzp"", Dict{Symbol, Any}(:name=>""numf_xzp"", :numcols=>Int64[]))])), 

In [7]:
function evaluate_pipeline(dfpipelines,X,Y;folds=3)
   res=@distributed (vcat) for prow in eachrow(dfpipelines)
      perf = crossvalidate(prow.Pipeline,X,Y;nfolds=folds)
      DataFrame(Description=prow.Description,mean=perf.mean,sd=perf.std)
   end
   return res
end

evaluate_pipeline (generic function with 1 method)

In [8]:
cross_res_oneblock = evaluate_pipeline(res_oneblock,X,Y)

      From worker 6:	fold: 1, 94.0
      From worker 6:	fold: 2, 94.0
      From worker 6:	fold: 3, 96.0
      From worker 6:	errors: 0
      From worker 5:	fold: 1, 92.0
      From worker 5:	fold: 2, 92.0
      From worker 5:	fold: 3, 90.0
      From worker 5:	errors: 0
      From worker 6:	fold: 1, 96.0
      From worker 6:	fold: 2, 96.0
      From worker 7:	fold: 1, 92.0
      From worker 6:	fold: 3, 96.0
      From worker 6:	errors: 0
      From worker 7:	fold: 2, 96.0
      From worker 7:	fold: 3, 94.0
      From worker 7:	errors: 0
      From worker 5:	fold: 1, 92.0
      From worker 5:	fold: 2, 94.0
      From worker 7:	fold: 1, 98.0
      From worker 7:	fold: 2, 96.0
      From worker 5:	fold: 3, 82.0
      From worker 7:	fold: 3, 98.0
      From worker 5:	errors: 0
      From worker 7:	errors: 0
      From worker 3:	fold: 1, 96.0
      From worker 4:	fold: 1, 98.0
      From worker 6:	fold: 1, 94.0
      From worker 6:	fold: 2, 98.0
      From worker 6:	fold: 3, 96.0
      Fro

      From worker 4:	fold: 3, 90.0
      From worker 4:	errors: 0
      From worker 2:	fold: 1, 96.0
      From worker 7:	fold: 3, 66.0
      From worker 7:	errors: 0
      From worker 3:	fold: 2, 98.0
      From worker 7:	fold: 1, 92.0
      From worker 9:	fold: 1, 86.0
      From worker 9:	fold: 2, 90.0
      From worker 9:	fold: 3, 92.0
      From worker 9:	errors: 0
      From worker 2:	fold: 2, 96.0
      From worker 7:	fold: 2, 90.0
      From worker 8:	fold: 1, 66.0
      From worker 4:	fold: 1, 92.0
      From worker 9:	fold: 1, 96.0
      From worker 9:	fold: 2, 90.0
      From worker 3:	fold: 3, 92.0
      From worker 3:	errors: 0
      From worker 9:	fold: 3, 86.0
      From worker 9:	errors: 0
      From worker 7:	fold: 3, 70.0
      From worker 7:	errors: 0
      From worker 2:	fold: 3, 98.0
      From worker 2:	errors: 0
      From worker 8:	fold: 2, 96.0
      From worker 7:	fold: 1, 82.0
      From worker 3:	fold: 1, 98.0
      From worker 8:	fold: 3, 88.0
      From wo

      From worker 3:	fold: 2, 98.0
      From worker 2:	fold: 2, 96.0
      From worker 4:	fold: 3, 96.0
      From worker 4:	errors: 0
      From worker 2:	fold: 3, 98.0
      From worker 2:	errors: 0
      From worker 3:	fold: 3, 96.0
      From worker 3:	errors: 0
      From worker 4:	fold: 1, 92.0
      From worker 2:	fold: 1, 96.0
      From worker 3:	fold: 1, 94.0
      From worker 4:	fold: 2, 98.0
      From worker 2:	fold: 2, 96.0
      From worker 3:	fold: 2, 94.0
      From worker 4:	fold: 3, 92.0
      From worker 4:	errors: 0
      From worker 2:	fold: 3, 94.0
      From worker 2:	errors: 0
      From worker 2:	fold: 1, 88.0
      From worker 3:	fold: 3, 94.0
      From worker 3:	errors: 0
      From worker 4:	fold: 1, 94.0
      From worker 2:	fold: 2, 100.0
      From worker 3:	fold: 1, 86.0
      From worker 4:	fold: 2, 96.0
      From worker 2:	fold: 3, 92.0
      From worker 2:	errors: 0
      From worker 3:	fold: 2, 90.0
      From worker 4:	fold: 3, 94.0
      From w

,Description,mean,sd
,String,Float64,Float64
1,(rb |> pca) |> rf,94.0,3.4641
2,(pt |> pca) |> rf,92.0,9.16515
3,(norm |> pca) |> rf,96.6667,2.3094
4,(mx |> pca) |> rf,95.3333,3.05505
5,(std |> pca) |> rf,96.6667,1.1547
6,(noop |> pca) |> rf,96.0,0.0
7,(rb |> fa) |> rf,90.0,5.2915
8,(pt |> fa) |> rf,90.0,2.0
9,(norm |> fa) |> rf,94.6667,2.3094


In [9]:
sort!(cross_res_oneblock,:mean, rev = true)

,Description,mean,sd
,String,Float64,Float64
1,(rb |> ica) |> rf,98.0,2.0
2,(norm |> ica) |> rbfsvc,98.0,2.0
3,(mx |> ica) |> rf,97.3333,2.3094
4,(norm |> noop) |> rf,97.3333,3.05505
5,(norm |> pca) |> gb,97.3333,2.3094
6,(mx |> pca) |> rbfsvc,97.3333,3.05505
7,(norm |> fa) |> rbfsvc,97.3333,1.1547
8,(pt |> noop) |> rbfsvc,97.3333,1.1547
9,(norm |> pca) |> ada,97.3333,2.3094


In [10]:
function twoblock_pipeline_factory(scalers,extractors,learners)
   results = @distributed (vcat) for lr in learners
      @distributed (vcat) for xt1 in extractors
         @distributed (vcat) for xt2 in extractors
            @distributed (vcat) for sc1 in scalers
               @distributed (vcat) for sc2 in scalers
                  prep = @pipeline ((catf |> ohe) + numf)
                  expx = @pipeline prep |> ((sc1 |> xt1) + (sc2 |> xt2)) |> lr
                  scn1   = sc1.name[1:end - 4];xtn1 = xt1.name[1:end - 4];
                  scn2   = sc2.name[1:end - 4];xtn2 = xt2.name[1:end - 4];
                  lrn = lr.name[1:end - 4]
                  pname = "($scn1 |> $xtn1) + ($scn2 |> $xtn2) |> $lrn"
                  DataFrame(Description=pname,Pipeline=expx)
               end
            end
         end
      end
   end
   return results
end
res_twoblock = twoblock_pipeline_factory(vscalers,vextractors,vlearners)

,Description,Pipeline
,String,Pipeline
1,(rb |> pca) + (rb |> pca) |> rf,"Pipeline(""linearpipeline_y9o"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_y9o"", :machines=>Machine[Pipeline(""linearpipeline_9ym"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_9ym"", :machines=>ComboPipeline[ComboPipeline(""combopipeline_ttT"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_ttT"", :machines=>Machine[Pipeline(""linearpipeline_IUL"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_IUL"", :machines=>Transformer[CatFeatureSelector(""catf_A2A"", Dict{Symbol, Any}(:name=>""catf_A2A"", :nominal_columns=>Int64[])), OneHotEncoder(""ohe_A0W"", Dict{Symbol, Any}(:nominal_column_values_map=>Dict{Int64, Any}(), :name=>""ohe_A0W"", :nominal_columns=>Int64[]))])), NumFeatureSelector(""numf_xzp"", Dict{Symbol, Any}(:name=>""numf_xzp"", :numcols=>Int64[]))])), ComboPipeline(""combopipeline_0eS"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_0eS"", :machines=>Pipeline[Pipeline(""linearpipeline_aHj"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_aHj"", :machines=>SKPreprocessor[SKPreprocessor(""rb_HXW"", Dict{Symbol, Any}(:name=>""rb_HXW"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""RobustScaler"")), SKPreprocessor(""pca_1Bi"", Dict{Symbol, Any}(:name=>""pca_1Bi"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PCA""))])), Pipeline(""linearpipeline_GIQ"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_GIQ"", :machines=>SKPreprocessor[SKPreprocessor(""rb_HXW"", Dict{Symbol, Any}(:name=>""rb_HXW"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""RobustScaler"")), SKPreprocessor(""pca_1Bi"", Dict{Symbol, Any}(:name=>""pca_1Bi"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PCA""))]))]))])), SKLearner(""rf_IBN"", Dict{Symbol, Any}(:output=>:class, :name=>""rf_IBN"", :impl_args=>Dict{Symbol, Any}(), :learner=>""RandomForestClassifier""))]))"
2,(rb |> pca) + (pt |> pca) |> rf,"Pipeline(""linearpipeline_M9C"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_M9C"", :machines=>Machine[Pipeline(""linearpipeline_pm6"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_pm6"", :machines=>ComboPipeline[ComboPipeline(""combopipeline_84b"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_84b"", :machines=>Machine[Pipeline(""linearpipeline_j3v"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_j3v"", :machines=>Transformer[CatFeatureSelector(""catf_A2A"", Dict{Symbol, Any}(:name=>""catf_A2A"", :nominal_columns=>Int64[])), OneHotEncoder(""ohe_A0W"", Dict{Symbol, Any}(:nominal_column_values_map=>Dict{Int64, Any}(), :name=>""ohe_A0W"", :nominal_columns=>Int64[]))])), NumFeatureSelector(""numf_xzp"", Dict{Symbol, Any}(:name=>""numf_xzp"", :numcols=>Int64[]))])), ComboPipeline(""combopipeline_Xmm"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_Xmm"", :machines=>Pipeline[Pipeline(""linearpipeline_CxF"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_CxF"", :machines=>SKPreprocessor[SKPreprocessor(""rb_HXW"", Dict{Symbol, Any}(:name=>""rb_HXW"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""RobustScaler"")), SKPreprocessor(""pca_1Bi"", Dict{Symbol, Any}(:name=>""pca_1Bi"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PCA""))])), Pipeline(""linearpipeline_BzV"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_BzV"", :machines=>SKPreprocessor[SKPreprocessor(""pt_IEv"", Dict{Symbol, Any}(:name=>""pt_IEv"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PowerTransformer"")), SKPreprocesso

In [11]:
function model_selection_pipeline(learners)
   results = @distributed (vcat) for lr in learners
      prep = @pipeline ((catf |> ohe) + numf)
      expx = @pipeline prep |> (rb |> pca)  |> lr
      pname = "(rb |> pca) |> $(lr.name[1:end-4])"
      DataFrame(Description=pname,Pipeline=expx)
   end
   return results
end
dfpipes = model_selection_pipeline(vlearners)

,Description,Pipeline
,String,Pipeline
1,(rb |> pca) |> rf,"Pipeline(""linearpipeline_1oJ"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_1oJ"", :machines=>Machine[Pipeline(""linearpipeline_qfH"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_qfH"", :machines=>Workflow[ComboPipeline(""combopipeline_876"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_876"", :machines=>Machine[Pipeline(""linearpipeline_Hjh"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_Hjh"", :machines=>Transformer[CatFeatureSelector(""catf_A2A"", Dict{Symbol, Any}(:name=>""catf_A2A"", :nominal_columns=>Int64[])), OneHotEncoder(""ohe_A0W"", Dict{Symbol, Any}(:nominal_column_values_map=>Dict{Int64, Any}(), :name=>""ohe_A0W"", :nominal_columns=>Int64[]))])), NumFeatureSelector(""numf_xzp"", Dict{Symbol, Any}(:name=>""numf_xzp"", :numcols=>Int64[]))])), Pipeline(""linearpipeline_vlK"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_vlK"", :machines=>SKPreprocessor[SKPreprocessor(""rb_HXW"", Dict{Symbol, Any}(:name=>""rb_HXW"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""RobustScaler"")), SKPreprocessor(""pca_1Bi"", Dict{Symbol, Any}(:name=>""pca_1Bi"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PCA""))]))])), SKLearner(""rf_IBN"", Dict{Symbol, Any}(:output=>:class, :name=>""rf_IBN"", :impl_args=>Dict{Symbol, Any}(), :learner=>""RandomForestClassifier""))]))"
2,(rb |> pca) |> gb,"Pipeline(""linearpipeline_iuH"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_iuH"", :machines=>Machine[Pipeline(""linearpipeline_7Rf"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_7Rf"", :machines=>Workflow[ComboPipeline(""combopipeline_cTS"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_cTS"", :machines=>Machine[Pipeline(""linearpipeline_TgO"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_TgO"", :machines=>Transformer[CatFeatureSelector(""catf_A2A"", Dict{Symbol, Any}(:name=>""catf_A2A"", :nominal_columns=>Int64[])), OneHotEncoder(""ohe_A0W"", Dict{Symbol, Any}(:nominal_column_values_map=>Dict{Int64, Any}(), :name=>""ohe_A0W"", :nominal_columns=>Int64[]))])), NumFeatureSelector(""numf_xzp"", Dict{Symbol, Any}(:name=>""numf_xzp"", :numcols=>Int64[]))])), Pipeline(""linearpipeline_0jo"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_0jo"", :machines=>SKPreprocessor[SKPreprocessor(""rb_HXW"", Dict{Symbol, Any}(:name=>""rb_HXW"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""RobustScaler"")), SKPreprocessor(""pca_1Bi"", Dict{Symbol, Any}(:name=>""pca_1Bi"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PCA""))]))])), SKLearner(""gb_uKa"", Dict{Symbol, Any}(:output=>:class, :name=>""gb_uKa"", :impl_args=>Dict{Symbol, Any}(), :learner=>""GradientBoostingClassifier""))]))"
3,(rb |> pca) |> lsvc,"Pipeline(""linearpipeline_KEZ"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_KEZ"", :machines=>Machine[Pipeline(""linearpipeline_e47"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_e47"", :machines=>Workflow[ComboPipeline(""combopipeline_nqN"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_nqN"", :machines=>Machine[Pipeline(""linearpipeline_RsW"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_RsW"", :machines=>Transformer[CatFeatureSelector(""catf_A2A"", Dict{Symbol, Any}(:name=>""catf_A2A"", :nominal_columns=>Int64[])), OneHotEncoder(""ohe_A0W"", Dict{Symbol, Any}(:nominal_column_values_map=>Dict{Int64, Any}(), :name=>""ohe_A0W"", :nominal_columns=>Int64[]))])), NumFeatureSelector(""numf_xzp"", Dict{Symbol, Any}(:name=>""numf_xzp"", :numcols=>Int64[]))])), 

In [12]:
# find the best model
@everywhere Random.seed!(10)
res = evaluate_pipeline(dfpipes,X,Y;folds=10)
sort!(res,:mean, rev = true)

      From worker 7:	fold: 1, 80.0
      From worker 4:	fold: 1, 86.66666666666667
      From worker 5:	fold: 1, 86.66666666666667
      From worker 7:	fold: 2, 100.0
      From worker 4:	fold: 2, 86.66666666666667
      From worker 7:	fold: 3, 93.33333333333333
      From worker 5:	fold: 2, 100.0
      From worker 4:	fold: 3, 100.0
      From worker 5:	fold: 3, 93.33333333333333
      From worker 7:	fold: 4, 86.66666666666667
      From worker 4:	fold: 4, 100.0
      From worker 7:	fold: 5, 93.33333333333333
      From worker 5:	fold: 4, 100.0
      From worker 4:	fold: 5, 86.66666666666667
      From worker 7:	fold: 6, 100.0
      From worker 5:	fold: 5, 100.0
      From worker 4:	fold: 6, 100.0
      From worker 7:	fold: 7, 93.33333333333333
      From worker 5:	fold: 6, 100.0
      From worker 4:	fold: 7, 93.33333333333333
      From worker 7:	fold: 8, 86.66666666666667
      From worker 5:	fold: 7, 93.33333333333333
      From worker 4:	fold: 8, 100.0
      From worker 7:	fold: 9,

,Description,mean,sd
,String,Float64,Float64
1,(rb |> pca) |> rbfsvc,96.0,4.66137
2,(rb |> pca) |> lsvc,94.0,5.8373
3,(rb |> pca) |> rf,93.3333,6.28539
4,(rb |> pca) |> gb,92.6667,3.78431
5,(rb |> pca) |> dt,92.0,6.12624
6,(rb |> pca) |> ada,74.0,12.7463


In [13]:
# use the best model to generate pipeline search
dfp = twoblock_pipeline_factory(vscalers,vextractors,[rbfsvc])

,Description,Pipeline
,String,Pipeline
1,(rb |> pca) + (rb |> pca) |> rbfsvc,"Pipeline(""linearpipeline_dfK"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_dfK"", :machines=>Machine[Pipeline(""linearpipeline_p5S"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_p5S"", :machines=>ComboPipeline[ComboPipeline(""combopipeline_i5g"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_i5g"", :machines=>Machine[Pipeline(""linearpipeline_9ls"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_9ls"", :machines=>Transformer[CatFeatureSelector(""catf_A2A"", Dict{Symbol, Any}(:name=>""catf_A2A"", :nominal_columns=>Int64[])), OneHotEncoder(""ohe_A0W"", Dict{Symbol, Any}(:nominal_column_values_map=>Dict{Int64, Any}(), :name=>""ohe_A0W"", :nominal_columns=>Int64[]))])), NumFeatureSelector(""numf_xzp"", Dict{Symbol, Any}(:name=>""numf_xzp"", :numcols=>Int64[]))])), ComboPipeline(""combopipeline_hLt"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_hLt"", :machines=>Pipeline[Pipeline(""linearpipeline_BMl"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_BMl"", :machines=>SKPreprocessor[SKPreprocessor(""rb_HXW"", Dict{Symbol, Any}(:name=>""rb_HXW"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""RobustScaler"")), SKPreprocessor(""pca_1Bi"", Dict{Symbol, Any}(:name=>""pca_1Bi"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PCA""))])), Pipeline(""linearpipeline_KDj"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_KDj"", :machines=>SKPreprocessor[SKPreprocessor(""rb_HXW"", Dict{Symbol, Any}(:name=>""rb_HXW"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""RobustScaler"")), SKPreprocessor(""pca_1Bi"", Dict{Symbol, Any}(:name=>""pca_1Bi"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PCA""))]))]))])), SKLearner(""rbfsvc_Du9"", Dict{Symbol, Any}(:output=>:class, :name=>""rbfsvc_Du9"", :impl_args=>Dict{Symbol, Any}(), :learner=>""SVC""))]))"
2,(rb |> pca) + (pt |> pca) |> rbfsvc,"Pipeline(""linearpipeline_28z"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_28z"", :machines=>Machine[Pipeline(""linearpipeline_hF7"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_hF7"", :machines=>ComboPipeline[ComboPipeline(""combopipeline_sbl"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_sbl"", :machines=>Machine[Pipeline(""linearpipeline_qxl"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_qxl"", :machines=>Transformer[CatFeatureSelector(""catf_A2A"", Dict{Symbol, Any}(:name=>""catf_A2A"", :nominal_columns=>Int64[])), OneHotEncoder(""ohe_A0W"", Dict{Symbol, Any}(:nominal_column_values_map=>Dict{Int64, Any}(), :name=>""ohe_A0W"", :nominal_columns=>Int64[]))])), NumFeatureSelector(""numf_xzp"", Dict{Symbol, Any}(:name=>""numf_xzp"", :numcols=>Int64[]))])), ComboPipeline(""combopipeline_5f7"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""combopipeline_5f7"", :machines=>Pipeline[Pipeline(""linearpipeline_3KS"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_3KS"", :machines=>SKPreprocessor[SKPreprocessor(""rb_HXW"", Dict{Symbol, Any}(:name=>""rb_HXW"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""RobustScaler"")), SKPreprocessor(""pca_1Bi"", Dict{Symbol, Any}(:name=>""pca_1Bi"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PCA""))])), Pipeline(""linearpipeline_KF1"", Dict{Symbol, Any}(:machine_args=>Dict{Symbol, Any}(), :name=>""linearpipeline_KF1"", :machines=>SKPreprocessor[SKPreprocessor(""pt_IEv"", Dict{Symbol, Any}(:name=>""pt_IEv"", :autocomponent=>false, :impl_args=>Dict{Any, Any}(), :preprocessor=>""PowerTransformer"")), SKPreprocessor("

In [14]:
# evaluate the pipeline
@everywhere Random.seed!(10)
bestp=evaluate_pipeline(dfp,X,Y;folds=3)

      From worker 2:	fold: 1, 94.0
      From worker 2:	fold: 2, 94.0
      From worker 3:	fold: 1, 94.0
      From worker 4:	fold: 1, 94.0
      From worker 6:	fold: 1, 94.0
      From worker 2:	fold: 3, 96.0
      From worker 2:	errors: 0
      From worker 8:	fold: 1, 94.0
      From worker 3:	fold: 2, 96.0
      From worker 5:	fold: 1, 96.0
      From worker 7:	fold: 1, 82.0
      From worker 4:	fold: 2, 92.0
      From worker 6:	fold: 2, 96.0
      From worker 8:	fold: 2, 94.0
      From worker 9:	fold: 1, 94.0
      From worker 4:	fold: 3, 96.0
      From worker 4:	errors: 0
      From worker 5:	fold: 2, 94.0
      From worker 8:	fold: 3, 96.0
      From worker 8:	errors: 0
      From worker 7:	fold: 2, 94.0
      From worker 2:	fold: 1, 100.0
      From worker 9:	fold: 2, 96.0
      From worker 8:	fold: 1, 100.0
      From worker 4:	fold: 1, 100.0
      From worker 2:	fold: 2, 96.0
      From worker 2:	fold: 3, 92.0
      From worker 8:	fold: 2, 96.0
      From worker 2:	errors: 

      From worker 4:	fold: 2, 96.0
      From worker 8:	fold: 3, 94.0
      From worker 8:	errors: 0
      From worker 5:	fold: 1, 94.0
      From worker 2:	fold: 1, 94.0
      From worker 8:	fold: 1, 94.0
      From worker 4:	fold: 3, 94.0
      From worker 4:	errors: 0
      From worker 9:	fold: 1, 96.0
      From worker 5:	fold: 2, 96.0
      From worker 8:	fold: 2, 96.0
      From worker 3:	fold: 1, 96.0
      From worker 4:	fold: 1, 90.0
      From worker 2:	fold: 2, 96.0
      From worker 7:	fold: 3, 96.0
      From worker 7:	errors: 0
      From worker 8:	fold: 3, 96.0
      From worker 8:	errors: 0
      From worker 6:	fold: 2, 90.0
      From worker 5:	fold: 3, 96.0
      From worker 5:	errors: 0
      From worker 8:	fold: 1, 96.0
      From worker 4:	fold: 2, 94.0
      From worker 2:	fold: 3, 96.0
      From worker 7:	fold: 1, 92.0
      From worker 8:	fold: 2, 96.0
      From worker 2:	errors: 0
      From worker 8:	fold: 3, 98.0
      From worker 2:	fold: 1, 96.0
      Fro

      From worker 2:	fold: 3, 92.0
      From worker 2:	errors: 0
      From worker 4:	fold: 1, 94.0
      From worker 8:	fold: 1, 94.0
      From worker 6:	fold: 2, 96.0
      From worker 9:	fold: 3, 96.0
      From worker 9:	errors: 0
      From worker 3:	fold: 2, 96.0
      From worker 4:	fold: 2, 98.0
      From worker 2:	fold: 1, 94.0
      From worker 5:	fold: 3, 100.0
      From worker 5:	errors: 0
      From worker 8:	fold: 2, 96.0
      From worker 4:	fold: 3, 94.0
      From worker 4:	errors: 0
      From worker 2:	fold: 2, 96.0
      From worker 3:	fold: 3, 96.0
      From worker 3:	errors: 0
      From worker 8:	fold: 3, 98.0
      From worker 8:	errors: 0
      From worker 4:	fold: 1, 96.0
      From worker 8:	fold: 1, 96.0
      From worker 2:	fold: 3, 96.0
      From worker 2:	errors: 0
      From worker 8:	fold: 2, 94.0
      From worker 2:	fold: 1, 98.0
      From worker 8:	fold: 3, 96.0
      From worker 8:	errors: 0
      From worker 8:	fold: 1, 92.0
      From worke

      From worker 4:	fold: 3, 98.0
      From worker 4:	errors: 0
      From worker 9:	fold: 1, 94.0
      From worker 6:	fold: 2, 98.0
      From worker 2:	fold: 2, 96.0
      From worker 5:	fold: 1, 96.0
      From worker 4:	fold: 1, 96.0
      From worker 6:	fold: 3, 96.0
      From worker 6:	errors: 0
      From worker 8:	fold: 2, 96.0
      From worker 3:	fold: 2, 98.0
      From worker 5:	fold: 2, 94.0
      From worker 6:	fold: 1, 96.0
      From worker 4:	fold: 2, 94.0
      From worker 2:	fold: 3, 94.0
      From worker 2:	errors: 0
      From worker 3:	fold: 3, 96.0
      From worker 3:	errors: 0
      From worker 7:	fold: 1, 92.0
      From worker 5:	fold: 3, 100.0
      From worker 4:	fold: 3, 96.0
      From worker 5:	errors: 0
      From worker 4:	errors: 0
      From worker 6:	fold: 2, 98.0
      From worker 6:	fold: 3, 96.0
      From worker 6:	errors: 0
      From worker 4:	fold: 1, 100.0
      From worker 5:	fold: 1, 96.0
      From worker 8:	fold: 3, 96.0
      From 

      From worker 3:	fold: 1, 90.0
      From worker 4:	fold: 1, 96.0
      From worker 6:	fold: 2, 98.0
      From worker 7:	fold: 1, 98.0
      From worker 2:	fold: 1, 94.0
      From worker 5:	fold: 3, 94.0
      From worker 8:	fold: 1, 100.0
      From worker 5:	errors: 0
      From worker 4:	fold: 2, 96.0
      From worker 2:	fold: 2, 94.0
      From worker 8:	fold: 2, 96.0
      From worker 9:	fold: 1, 92.0
      From worker 4:	fold: 3, 90.0
      From worker 8:	fold: 3, 90.0
      From worker 2:	fold: 3, 92.0
      From worker 4:	errors: 0
      From worker 8:	errors: 0
      From worker 2:	errors: 0
      From worker 5:	fold: 1, 94.0
      From worker 2:	fold: 1, 100.0
      From worker 8:	fold: 1, 96.0
      From worker 5:	fold: 2, 98.0
      From worker 2:	fold: 2, 98.0
      From worker 8:	fold: 2, 96.0
      From worker 3:	fold: 2, 100.0
      From worker 4:	fold: 1, 96.0
      From worker 2:	fold: 3, 90.0
      From worker 2:	errors: 0
      From worker 5:	fold: 3, 96.0
  

...

      From worker 9:	fold: 1, 98.0
      From worker 3:	fold: 1, 98.0
      From worker 4:	fold: 2, 90.0
      From worker 3:	fold: 2, 94.0
      From worker 9:	fold: 2, 98.0
      From worker 3:	fold: 3, 96.0
      From worker 3:	errors: 0
      From worker 4:	fold: 3, 96.0
      From worker 4:	errors: 0
      From worker 6:	fold: 2, 96.0
      From worker 3:	fold: 1, 96.0
      From worker 5:	fold: 3, 94.0
      From worker 5:	errors: 0
      From worker 3:	fold: 2, 96.0
      From worker 9:	fold: 3, 94.0
      From worker 9:	errors: 0
      From worker 3:	fold: 3, 92.0
      From worker 3:	errors: 0
      From worker 4:	fold: 1, 92.0
      From worker 3:	fold: 1, 98.0
      From worker 9:	fold: 1, 98.0
      From worker 7:	fold: 1, 84.0
      From worker 4:	fold: 2, 92.0
      From worker 6:	fold: 3, 94.0
      From worker 6:	errors: 0
      From worker 5:	fold: 1, 98.0
      From worker 3:	fold: 2, 98.0
      From worker 9:	fold: 2, 96.0
      From worker 4:	fold: 3, 92.0
      Fro

      From worker 9:	fold: 3, 94.0
      From worker 9:	errors: 0
      From worker 6:	fold: 2, 96.0
      From worker 9:	fold: 1, 96.0
      From worker 9:	fold: 2, 100.0
      From worker 9:	fold: 3, 96.0
      From worker 9:	errors: 0
      From worker 9:	fold: 1, 94.0
      From worker 9:	fold: 2, 96.0
      From worker 5:	fold: 3, 96.0
      From worker 5:	errors: 0
      From worker 7:	fold: 3, 94.0
      From worker 7:	errors: 0
      From worker 9:	fold: 3, 100.0
      From worker 9:	errors: 0
      From worker 9:	fold: 1, 98.0
      From worker 9:	fold: 2, 94.0
      From worker 5:	fold: 1, 90.0
      From worker 9:	fold: 3, 96.0
      From worker 9:	errors: 0
      From worker 6:	fold: 3, 94.0
      From worker 6:	errors: 0
      From worker 5:	fold: 2, 94.0
      From worker 9:	fold: 1, 98.0
      From worker 9:	fold: 2, 94.0
      From worker 5:	fold: 3, 98.0
      From worker 5:	errors: 0
      From worker 7:	fold: 1, 96.0
      From worker 3:	fold: 2, 96.0
      From work

      From worker 5:	fold: 3, 94.0
      From worker 5:	errors: 0
      From worker 6:	fold: 3, 98.0
      From worker 6:	errors: 0
      From worker 5:	fold: 1, 92.0
      From worker 6:	fold: 1, 96.0
      From worker 5:	fold: 2, 88.0
      From worker 6:	fold: 2, 88.0
      From worker 5:	fold: 3, 98.0
      From worker 5:	errors: 0
      From worker 5:	fold: 1, 96.0
      From worker 6:	fold: 3, 100.0
      From worker 6:	errors: 0
      From worker 5:	fold: 2, 96.0
      From worker 5:	fold: 3, 94.0
      From worker 5:	errors: 0
      From worker 5:	fold: 1, 96.0
      From worker 7:	fold: 3, 98.0
      From worker 7:	errors: 0
      From worker 6:	fold: 1, 94.0
      From worker 5:	fold: 2, 92.0
      From worker 5:	fold: 3, 98.0
      From worker 7:	fold: 1, 98.0
      From worker 5:	errors: 0
      From worker 5:	fold: 1, 96.0
      From worker 6:	fold: 2, 98.0
      From worker 5:	fold: 2, 98.0
      From worker 5:	fold: 3, 96.0
      From worker 5:	errors: 0
      From worke

      From worker 6:	fold: 2, 92.0
      From worker 7:	fold: 1, 98.0
      From worker 6:	fold: 3, 96.0
      From worker 6:	errors: 0
      From worker 7:	fold: 2, 98.0
      From worker 6:	fold: 1, 96.0
      From worker 6:	fold: 2, 96.0
      From worker 6:	fold: 3, 98.0
      From worker 6:	errors: 0
      From worker 7:	fold: 3, 94.0
      From worker 7:	errors: 0
      From worker 6:	fold: 1, 96.0
      From worker 6:	fold: 2, 96.0
      From worker 6:	fold: 3, 94.0
      From worker 6:	errors: 0
      From worker 6:	fold: 1, 96.0
      From worker 7:	fold: 1, 98.0
      From worker 6:	fold: 2, 94.0
      From worker 6:	fold: 3, 100.0
      From worker 6:	errors: 0
      From worker 7:	fold: 2, 96.0
      From worker 6:	fold: 1, 92.0
      From worker 6:	fold: 2, 98.0
      From worker 6:	fold: 3, 98.0
      From worker 6:	errors: 0
      From worker 7:	fold: 3, 96.0
      From worker 7:	errors: 0
      From worker 7:	fold: 1, 96.0
      From worker 6:	fold: 1, 94.0
      From w

,Description,mean,sd
,String,Float64,Float64
1,(rb |> pca) + (rb |> pca) |> rbfsvc,94.6667,1.1547
2,(rb |> pca) + (pt |> pca) |> rbfsvc,96.0,4.0
3,(rb |> pca) + (norm |> pca) |> rbfsvc,95.3333,2.3094
4,(rb |> pca) + (mx |> pca) |> rbfsvc,94.6667,3.05505
5,(rb |> pca) + (std |> pca) |> rbfsvc,96.6667,3.05505
6,(rb |> pca) + (noop |> pca) |> rbfsvc,94.6667,4.16333
7,(pt |> pca) + (rb |> pca) |> rbfsvc,95.3333,2.3094
8,(pt |> pca) + (pt |> pca) |> rbfsvc,95.3333,3.05505
9,(pt |> pca) + (norm |> pca) |> rbfsvc,96.6667,2.3094


In [15]:
sort!(bestp,:mean, rev = true)

,Description,mean,sd
,String,Float64,Float64
1,(noop |> pca) + (mx |> noop) |> rbfsvc,99.3333,1.1547
2,(noop |> pca) + (noop |> ica) |> rbfsvc,98.6667,1.1547
3,(norm |> fa) + (rb |> pca) |> rbfsvc,98.6667,1.1547
4,(norm |> fa) + (mx |> pca) |> rbfsvc,98.6667,1.1547
5,(mx |> fa) + (norm |> fa) |> rbfsvc,98.6667,2.3094
6,(std |> ica) + (norm |> pca) |> rbfsvc,98.6667,1.1547
7,(std |> ica) + (noop |> pca) |> rbfsvc,98.6667,2.3094
8,(noop |> ica) + (noop |> pca) |> rbfsvc,98.6667,1.1547
9,(mx |> ica) + (norm |> fa) |> rbfsvc,98.6667,1.1547


In [16]:
show(bestp;allrows=true,truncate=0,allcols=true)

576×3 DataFrame
 Row │ Description                                mean     sd      
     │ String                                     Float64  Float64 
─────┼─────────────────────────────────────────────────────────────
   1 │ (noop |> pca) + (mx |> noop) |> rbfsvc     99.3333  1.1547
   2 │ (noop |> pca) + (noop |> ica) |> rbfsvc    98.6667  1.1547
   3 │ (norm |> fa) + (rb |> pca) |> rbfsvc       98.6667  1.1547
   4 │ (norm |> fa) + (mx |> pca) |> rbfsvc       98.6667  1.1547
   5 │ (mx |> fa) + (norm |> fa) |> rbfsvc        98.6667  2.3094
   6 │ (std |> ica) + (norm |> pca) |> rbfsvc     98.6667  1.1547
   7 │ (std |> ica) + (noop |> pca) |> rbfsvc     98.6667  2.3094
   8 │ (noop |> ica) + (noop |> pca) |> rbfsvc    98.6667  1.1547
   9 │ (mx |> ica) + (norm |> fa) |> rbfsvc       98.6667  1.1547
  10 │ (noop |> pca) + (norm |> pca) |> rbfsvc    98.0     0.0
  11 │ (noop |> pca) + (mx |> pca) |> rbfsvc      98.0     2.0
  12 │ (noop |> pca) + (noop |> pca) |> rbfsvc    98.0     0